In [1]:
import torch
import os
from torch.utils.data import DataLoader
from dataset.dataset import get_cdiscount_dataset
from model.model import assemble_model
from predictor.predictor import get_predictor

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
config = {
    'test_batch_size': 1000,
    'arch': 'resnet34',
    'checkpoint': 'resnet34_checkpoint_epoch_0_iter_3000.pth.tar',
    'print_freq': 10,
    'pred_filename': "Resnet34-Epoch_0-iter-3000.csv"
}

In [ ]:
import torchvision.models as models

# get dataset
print('getting dataset...')
test_dataset = get_cdiscount_dataset(offsets_csv="test_offsets.csv",
                                     images_csv="test_images.csv",
                                     bson_file_path="/mnt/data/cdiscount/test.bson",
                                     with_label=False,
                                     resize=256)

# get data loader
print('getting data loader...')
test_dataloader = DataLoader(test_dataset, batch_size=config['test_batch_size'], shuffle=False, num_workers=6)

# define model
print("=> using pre-trained model '{}'".format(config['arch']))
model = models.__dict__[config['arch']](pretrained=True)
model = assemble_model(model, -1, 512, 5270)
model = torch.nn.DataParallel(model).cuda()


# load checkpoint
if not os.path.isfile(config['checkpoint']):
    print("=> no checkpoint found at '{}'".format(config['checkpoint']))
    
print("=> loading checkpoint '{}'".format(config['checkpoint']))
checkpoint = torch.load(config['checkpoint'])
model.load_state_dict(checkpoint['state_dict'])
print("=> loaded checkpoint:")

print('Epoch: [{0}][{1}]\t'
      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
      'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
       checkpoint['epoch'], checkpoint['iter'], loss=checkpoint['loss'], top1=checkpoint['top1'], top5=checkpoint['top5']))

# get trainer
Predictor = get_predictor(test_dataloader, model, config)

# Run!
Predictor.run()

getting dataset...
getting data loader...
=> using pre-trained model 'resnet34'
=> loading checkpoint 'resnet34_checkpoint_epoch_0_iter_3000.pth.tar'
=> loaded checkpoint:
Epoch: [0][3000]	Loss 3.0118 (3.5749)	Prec@1 44.922 (40.549)	Prec@5 64.844 (57.848)
start prediction
Iter: [0/3096]	Time 31.179	Data 29.271	
Iter: [10/3096]	Time 2.702	Data 0.014	
Iter: [20/3096]	Time 2.979	Data 0.010	
Iter: [30/3096]	Time 17.188	Data 15.086	
Iter: [40/3096]	Time 2.792	Data 0.017	
Iter: [50/3096]	Time 2.974	Data 0.001	
Iter: [60/3096]	Time 16.912	Data 14.708	
